# Capstone Project - The Battle of the Neighborhoods (Week 2)



## Introduction: Business Problem <a name="introduction"></a>

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening an **Chinese restaurant** in **Auckland**, New Zealand.

Since there are lots of restaurants in Auckland, especially in Auckland city center, we will try to detect **locations that are not already crowded with restaurants**. We are also particularly interested in **areas with no Chinese restaurants in vicinity**. We would also prefer locations **as close to city center as possible**, assuming that first two conditions are met.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of existing restaurants in the neighborhood (any type of restaurant)
* number of and distance to Italian restaurants in the neighborhood, if any
* distance of neighborhood from city center

We decided to use regularly spaced grid of locations, centered around city center, to define our neighborhoods.

Following data sources will be needed to extract/generate the required information:
* centers of candidate areas will be generated algorithmically and approximate addresses of centers of those areas will be obtained using **Google Maps API reverse geocoding**
* number of restaurants and their type and location in every neighborhood will be obtained using **Foursquare API**
* coordinate of Auckland center will be obtained using **Google Maps API geocoding** of well known Auckland location (Alexanderplatz)

### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 12x12 killometers centered around Auckland city center.

Let's first find the latitude & longitude of Auckland city center, using specific, well known address and Google Maps geocoding API.

In [9]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup


def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
address = 'Sky Tower, Auckland, New Zealand'

Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~6km from Sky Tower. Our neighborhoods will be defined as circular areas with a radius of 300 meters, so our neighborhood centers will be 600 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in  meters).

In [10]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Auckland center longitude={}, latitude={}'.format(Auckland_center[1], Auckland_center[0]))
x, y = lonlat_to_xy(Auckland[1], Auckland_center[0])
print('Auckland center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Auckland center longitude={}, latitude={}'.format(lo, la))

ModuleNotFoundError: No module named 'shapely'

Let's create a **hexagonal grid of cells**: we offset every other row, and adjust vertical row spacing so that **every cell center is equally distant from all it's neighbors**.

In [6]:
Auckland_center_x, Auckland_center_y = lonlat_to_xy(Auckland_center[1], bAuckland_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = Auckland_center_x - 6000
x_step = 600
y_min = Auckland_center_y - 6000 - (int(21/k)*k*600 - 12000)/2
y_step = 600 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 300 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(Auckland_center_x, Auckland_center_y, x, y)
        if (distance_from_center <= 6001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [9]:
#!pip install folium

import folium

In [11]:
map_Auckland = folium.Map(location=Auckland_center, zoom_start=13)
folium.Marker(Auckland_center, popup='Sky Tower').add_to(map_Auckland)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_Auckland) 
    folium.Circle([lat, lon], radius=300, color='blue', fill=False).add_to(map_Auckland)
    #folium.Marker([lat, lon]).add_to(map_Auckland)
map_Auckland

NameError: name 'folium' is not defined

Let's now use Google Maps API to get approximate addresses of those locations.

In [12]:

def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, Auckland_center[0], Auckland_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(Auckland_center[0], Auckland_center[1], addr))

NameError: name 'google_api_key' is not defined

Looking good. Let's now place all this into a Pandas dataframe.

In [13]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

NameError: name 'addresses' is not defined

...and let's now save/persist this data into local file.

In [20]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Italian restaurants:', len(italian_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(italian_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 2031
Total number of Italian restaurants: 312
Percentage of Italian restaurants: 15.36%
Average number of restaurants in neighborhood: 4.91208791209


Let's now see all the collected restaurants in our area of interest on map, and let's also show Italian restaurants in different color.

In [29]:
map_Auckland = folium.Map(location=Auckland_center, zoom_start=13)
folium.Marker(Auckland_center, popup='Sky Tower').add_to(map_Auckland)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_italian = res[6]
    color = 'red' if is_italian else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_Auckland)
map_Auckland

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the **number of restaurants in every area candidate**:

In [2]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

NameError: name 'location_restaurants' is not defined

OK, now let's calculate the **distance to nearest Italian restaurant from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [14]:
distances_to_italian_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 10000
    for res in italian_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_italian_restaurant.append(min_distance)

df_locations['Distance to Italian restaurant'] = distances_to_italian_restaurant

NameError: name 'xs' is not defined

In [15]:
df_locations.head(10)

NameError: name 'df_locations' is not defined

In [35]:
print('Average distance to closest Italian restaurant from each area center:', df_locations['Distance to Italian restaurant'].mean())

Average distance to closest Italian restaurant from each area center: 495.2099580523902


OK, so **on average Italian restaurant can be found within ~500m** from every area center candidate. That's fairly close, so we need to filter our areas carefully!

In [38]:
from folium import plugins
from folium.plugins import HeatMap

map_Auckland = folium.Map(location=Auckland_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_Auckland) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_Auckland)
folium.Marker(Auckland_center).add_to(map_Auckland)
folium.Circle(Auckland_center, radius=1000, fill=False, color='white').add_to(map_Auckland)
folium.Circle(Auckland_center, radius=2000, fill=False, color='white').add_to(map_Auckland)
folium.Circle(Auckland_center, radius=3000, fill=False, color='white').add_to(map_Auckland)
folium.GeoJson(Auckland_boroughs, style_function=boroughs_style, name='geojson').add_to(map_Auckland)
map_Auckland

In [39]:
map_Auckland = folium.Map(location=Auckland_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_Auckland) #cartodbpositron cartodbdark_matter
HeatMap(italian_latlons).add_to(map_Auckland)
folium.Marker(Auckland_center).add_to(map_Auckland)
folium.Circle(Auckland_center, radius=1000, fill=False, color='white').add_to(map_Auckland)
folium.Circle(Auckland_center, radius=2000, fill=False, color='white').add_to(map_Auckland)
folium.Circle(Auckland_center, radius=3000, fill=False, color='white').add_to(map_Auckland)
folium.GeoJson(Auckland_boroughs, style_function=boroughs_style, name='geojson').add_to(map_Auckland)
map_Auckland

In [40]:
roi_x_min = Auckland_x - 2000
roi_y_max = Auckland_y + 1000
roi_width = 5000
roi_height = 5000
roi_center_x = roi_x_min + 2500
roi_center_y = roi_y_max - 2500
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_Auckland = folium.Map(location=roi_center, zoom_start=14)
HeatMap(restaurant_latlons).add_to(map_Auckland)
folium.Marker(Auckland_center).add_to(map_Auckland)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_Auckland)
folium.GeoJson(bAuckland_boroughs, style_function=boroughs_style, name='geojson').add_to(map_Auckland)
map_Auckland